In [ ]:
#nsa
##we will start prediction pipeline

import pandas as pd


current_loans = pd.read_csv("./extracted_files//current_loans.csv")
current_loans.info()

sample_current_loans = current_loans.sample(100)

target = "loan_status"
prediction = "prediction"
numerical_features = ["funded_amnt", "int_rate", "annual_inc", "dti", "fico_range_high", "revol_bal", "revol_util", "bc_util", "total_bal_ex_mort", "inq_last_12m", "num_accts_ever_120_pd"]
categorical_features = ["term", "home_ownership", "delinq_2yrs", "mort_acc"]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251552 entries, 0 to 251551
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      251552 non-null  int64  
 1   funded_amnt             251552 non-null  int64  
 2   term                    251552 non-null  object 
 3   int_rate                251552 non-null  object 
 4   installment             251552 non-null  float64
 5   emp_title               233760 non-null  object 
 6   emp_length              233983 non-null  object 
 7   home_ownership          251552 non-null  object 
 8   annual_inc              251552 non-null  float64
 9   loan_status             251552 non-null  object 
 10  purpose                 251552 non-null  object 
 11  zip_code                251552 non-null  object 
 12  addr_state              251552 non-null  object 
 13  dti                     251358 non-null  float64
 14  delinq_2yrs         

In [41]:
df = pd.concat([sample_current_loans[numerical_features + categorical_features]], axis=1, ignore_index=False)
df.info()
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 119384 to 235232
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   funded_amnt            100 non-null    int64  
 1   int_rate               100 non-null    object 
 2   annual_inc             100 non-null    float64
 3   dti                    100 non-null    float64
 4   fico_range_high        100 non-null    int64  
 5   revol_bal              100 non-null    int64  
 6   revol_util             100 non-null    object 
 7   bc_util                100 non-null    float64
 8   total_bal_ex_mort      100 non-null    int64  
 9   inq_last_12m           100 non-null    float64
 10  num_accts_ever_120_pd  100 non-null    int64  
 11  term                   100 non-null    object 
 12  home_ownership         100 non-null    object 
 13  delinq_2yrs            100 non-null    int64  
 14  mort_acc               100 non-null    int64  
dtypes: 

In [42]:
import numpy as np
## data preprocessing

#int_rate we have to remove % sign so that we can 
df["int_rate"] = df["int_rate"].apply(lambda x: x.replace('%', ''))
df['int_rate'] = df['int_rate'].astype('float64')


In [43]:
#revol_util
df['revol_util'] = df['revol_util'].apply(lambda x: 0 if x == 'nan' else x)
df['revol_util'] = df['revol_util'].apply(lambda x: str(x).replace('%', ''))
df['revol_util'] = df['revol_util'].astype('float64')

df['revol_util'].value_counts().unique

<bound method Series.unique of revol_util
33.6    2
42.8    2
81.1    2
86.7    2
39.8    2
       ..
59.3    1
68.1    1
45.8    1
27.9    1
76.7    1
Name: count, Length: 95, dtype: int64>

In [44]:
##here categorical data will start

#term
df['term'] = df['term'].apply(lambda x: 0 if x=='36 months' else 1)
df['term'].astype('Int32')

119384    1
97002     1
147249    1
41756     1
98706     1
         ..
10546     1
48760     1
139503    1
197199    1
235232    1
Name: term, Length: 100, dtype: Int32

In [45]:
#home_ownership ['RENT', 'MORTGAGE', 'OWN', 'ANY'] => [0,1,2,3]
df['home_ownership'] = df['home_ownership'].replace(['RENT', 'MORTGAGE', 'OWN', 'ANY','NONE'], [0,1,2,3,3])

# df['home_ownership'] = df['home_ownership'].astype('Int32')
df['home_ownership'].value_counts().unique
# df.info()

C:\Users\ujjwa\AppData\Local\Temp\ipykernel_7452\3670027532.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['home_ownership'] = df['home_ownership'].replace(['RENT', 'MORTGAGE', 'OWN', 'ANY','NONE'], [0,1,2,3,3])


<bound method Series.unique of home_ownership
1    44
0    43
2    13
Name: count, dtype: int64>

In [46]:
df['delinq_2yrs'].unique()

array([0, 1, 5, 4, 2])

In [47]:
# inq_last_12m Numeric feature
df.replace(np.nan, 0, inplace=True)
df['inq_last_12m'] = df['inq_last_12m'].astype('int64')

df['inq_last_12m'].value_counts().unique

<bound method Series.unique of inq_last_12m
1     26
0     23
2     15
3     13
4     10
5      4
6      4
8      2
10     1
7      1
11     1
Name: count, dtype: int64>

In [48]:
print(df.shape)
df.head()
df.info()

(100, 15)
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 119384 to 235232
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   funded_amnt            100 non-null    int64  
 1   int_rate               100 non-null    float64
 2   annual_inc             100 non-null    float64
 3   dti                    100 non-null    float64
 4   fico_range_high        100 non-null    int64  
 5   revol_bal              100 non-null    int64  
 6   revol_util             100 non-null    float64
 7   bc_util                100 non-null    float64
 8   total_bal_ex_mort      100 non-null    int64  
 9   inq_last_12m           100 non-null    int64  
 10  num_accts_ever_120_pd  100 non-null    int64  
 11  term                   100 non-null    int64  
 12  home_ownership         100 non-null    int64  
 13  delinq_2yrs            100 non-null    int64  
 14  mort_acc               100 non-null    int64 

In [49]:
df['dti'].isnull().sum()

np.int64(0)

In [50]:
import pickle

# Open the pickle file in binary read mode
with open('../artifacts/preprocessing_object.pickle', 'rb') as file:
    # Load the data from the pickle file
    preprocessor = pickle.load(file)

In [51]:
prediction_features = preprocessor.transform(df)


In [52]:
prediction_features

array([[ 2.36778917,  0.49392238,  0.2684648 , ...,  2.77559937,
         0.        ,  0.        ],
       [ 1.79674532,  2.72173962, -0.00963931, ...,  0.        ,
         0.        ,  0.        ],
       [-0.48743007,  0.49392238, -0.56584753, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.7158476 , -0.33817688, -0.03744972, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08361378,  0.51296356,  0.7412418 , ...,  2.77559937,
         0.        ,  0.        ],
       [-1.28689145, -0.45813627, -1.26110781, ...,  0.        ,
         0.        ,  0.        ]])

In [53]:
# Open the pickle file in binary read mode
with open('../artifacts/best_model.pickle', 'rb') as file:
    # Load the data from the pickle file
    best_model = pickle.load(file)
    

In [54]:
predicted_probabilities = best_model.predict_proba(prediction_features)[:,1] # this is for the class 1
predicted_probabilities

array([0.45030265, 0.38691123, 0.67404018, 0.52705303, 0.49734328,
       0.37407197, 0.40393764, 0.72992358, 0.78190197, 0.41304206,
       0.92567541, 0.70187274, 0.66131607, 0.22597618, 0.36298971,
       0.65973264, 0.57005523, 0.22403524, 0.45461353, 0.51312709,
       0.74124292, 0.70528945, 0.51026045, 0.41397242, 0.545644  ,
       0.68673365, 0.86882244, 0.42659503, 0.70289679, 0.69970084,
       0.77269093, 0.53336942, 0.25521939, 0.71378966, 0.62853681,
       0.75622422, 0.5533958 , 0.75779869, 0.69214649, 0.76687232,
       0.26386418, 0.43499111, 0.7101492 , 0.66509211, 0.59512001,
       0.76843815, 0.81777522, 0.55942751, 0.14045883, 0.49062272,
       0.45129506, 0.60091246, 0.43413408, 0.39177307, 0.34043752,
       0.62527334, 0.49495592, 0.43375828, 0.56636375, 0.78683466,
       0.57033863, 0.78795936, 0.53773689, 0.77101411, 0.68692437,
       0.62709149, 0.30587577, 0.67105864, 0.5302305 , 0.68584072,
       0.36731714, 0.3536338 , 0.66780914, 0.35259411, 0.74563

In [55]:
sample_current_loans['investments'] = 0
sample_current_loans['predictions'] = predicted_probabilities
sample_current_loans.to_csv("./extracted_files/sample_current_loans_with_predictions.csv", index=False)

In [56]:
import pandas as pd
df = pd.read_csv('./extracted_files/sample_current_loans_with_predictions.csv')
df.head()

,id,funded_amnt,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,loan_status,...,dti_joint,inq_last_12m,acc_open_past_24mths,bc_util,mort_acc,num_accts_ever_120_pd,percent_bc_gt_75,total_bal_ex_mort,investments,predictions
0,96414500,35000,36 months,16.99%,1247.68,Account Manager,< 1 year,MORTGAGE,80000.0,Current,...,NaN,0.0,5,51.1,2,0,33.3,28617,0,0.450303
1,93532550,30000,60 months,28.69%,946.61,Rig Supervisor,8 years,MORTGAGE,70000.0,Current,...,NaN,4.0,2,55.8,1,0,0.0,37229,0,0.386911
2,105537312,10000,36 months,16.99%,356.48,Carpenter,10+ years,OWN,50000.0,Current,...,NaN,5.0,9,38.5,1,0,25.0,33971,0,0.674040
3,76372781,18275,60 months,28.18%,571.00,Assistant product manager,< 1 year,OWN,65000.0,Current,...,NaN,4.0,12,90.9,1,0,100.0,75499,0,0.527053
4,97414793,35000,60 months,29.99%,1132.16,Computer / Compliance,10+ years,MORTGAGE,98000.0,Current,...,NaN,6.0,9,47.6,1,0,33.3,84759,0,0.497343


In [57]:
df = df[['id', 'predictions', 'funded_amnt', 'term', 'int_rate', 'investments']]
df.head()

,id,predictions,funded_amnt,term,int_rate,investments
0,96414500,0.450303,35000,36 months,16.99%,0
1,93532550,0.386911,30000,60 months,28.69%,0
2,105537312,0.674040,10000,36 months,16.99%,0
3,76372781,0.527053,18275,60 months,28.18%,0
4,97414793,0.497343,35000,60 months,29.99%,0
